# How to get MOC data from VESPA in Python

Illustrate search for TAP MOC for vespa using 'pyvo' and 'astropy'

## Import required modules

In [2]:
import pyvo as vo
from mpl_toolkits.basemap import Basemap

## Search a specific TAP service

We are looking for VVEx - VIRTIS/Venus Express database since it has a MOC

### Search the database containing this service

In [3]:
keywords = ["Venus"]

In [4]:
databasesList = vo.registry.search(servicetype='tap',keywords=keywords, includeaux=True)

In [5]:
for d in databasesList :
    print(d.short_name)

J/A+A/537/L2
J/A+A/585/A53
J/A+A/606/A53
J/A+A/613/A25
J/A+A/619/A99
J/A+A/627/A82
J/A+A/653/A41
J/AJ/154/207
J/AJ/158/196
J/AJ/160/131
J/AJ/160/259
J/AJ/162/216
J/ApJ/800/135
J/ApJ/807/45
J/ApJ/893/L27
J/MNRAS/462/4300
J/MNRAS/485/3999
J/MNRAS/519/6210
VI/127

GRB/IPN
KONUS
Cassini RPWS
BDIP
bdip.observatori
HST Planeto
VVEx
MESS-MAG-VSO
MESS-MAG-VSO-AVG
PVO-PHEM-ASC
PVO-PHEM-BIN
VEX-ASPERA4-DER
VEX-ASPERA4-ENG
VEX-ASPERA4-RAW
VEX-HIGH-MAG


### Search the services in the databases that were found

In [6]:
urlsList = set(databasesList.to_table().field("access_urls"))

In [7]:
tables = []

for url in urlsList :
    print("Searching in service " + url)
    service = vo.dal.TAPService(url)
    searchResult = service.search("SELECT schema_name, table_name FROM tap_schema.tables WHERE schema_name LIKE '%"+keywords[0]+"%' OR table_name LIKE '%"+keywords[0]+"%' OR description LIKE '%"+keywords[0]+"%'")
    for tableFound in searchResult["table_name"] :
        tables.append({"url":url,"name":tableFound})
print("DONE")

Searching in service http://voparis-tap-planeto.obspm.fr/tap
Searching in service http://vo.lmd.jussieu.fr/tap
Searching in service http://voparis-tap-maser.obspm.fr/tap
Searching in service https://vo-pds-ppi.igpp.ucla.edu/tap
Searching in service http://tapvizier.cds.unistra.fr/TAPVizieR/tap
Searching in service https://heasarc.gsfc.nasa.gov/xamin/vo/tap
DONE


In [8]:
for table in tables :
    print(table["name"]," at ", table["url"])

vvex.epn_core  at  http://voparis-tap-planeto.obspm.fr/tap
vex_cleaned_high_res_mag.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
vex_aspera4_els_raw.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
mess_mag_calibrated_vso.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
vex_aspera4_els_derived.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
mess_mag_calibrated_vso_avg.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
pvo_ephem_vso_asc.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
vex_aspera4_els_eng.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
pvo_ephem_vso_bin.epn_core  at  https://vo-pds-ppi.igpp.ucla.edu/tap
J/A+A/627/A82/tablea1  at  http://tapvizier.cds.unistra.fr/TAPVizieR/tap
J/A+A/650/A156/tablec1  at  http://tapvizier.cds.unistra.fr/TAPVizieR/tap
J/A+A/627/A82/tablea3  at  http://tapvizier.cds.unistra.fr/TAPVizieR/tap
J/A+A/650/A156/table2  at  http://tapvizier.cds.unistra.fr/TAPVizieR/tap
J/A+A/537/L2/tablea1  at  http://tapvizier.cds.unistra.fr/TAPVi

## Only keep tables with a 'coverage' key (and so maybe a MOC)

In [9]:
def doContainMOC(table) :
    service = vo.dal.TAPService(table["url"])
    try :
        searchResult = service.search("SELECT TOP 1 * FROM "+table["name"]+" WHERE coverage IS NOT NULL")
        if searchResult == "" :
            return False
        else :
            return True
    except :
        return False

In [10]:
tablesWithMOC = []
for table in tables :
    print("Checking table ",table["name"])
    if doContainMOC(table) :
        tablesWithMOC.append(table)

Checking table  vvex.epn_core
Checking table  vex_cleaned_high_res_mag.epn_core
Checking table  vex_aspera4_els_raw.epn_core
Checking table  mess_mag_calibrated_vso.epn_core
Checking table  vex_aspera4_els_derived.epn_core
Checking table  mess_mag_calibrated_vso_avg.epn_core
Checking table  pvo_ephem_vso_asc.epn_core
Checking table  vex_aspera4_els_eng.epn_core
Checking table  pvo_ephem_vso_bin.epn_core
Checking table  J/A+A/627/A82/tablea1
Checking table  J/A+A/650/A156/tablec1
Checking table  J/A+A/627/A82/tablea3
Checking table  J/A+A/650/A156/table2
Checking table  J/A+A/537/L2/tablea1
Checking table  J/A+A/627/A82/tablea2
Checking table  J/AJ/162/216/fig11


In [11]:
for table in tablesWithMOC :
    print(table["name"])

vvex.epn_core


## Display the first 10 elements in the table
Here we only consider the first table found in the previous section

In [12]:
def getData(tapService, serviceStructure, query) :
    if doContainMOC(serviceStructure) :
        return tapService.search(query).to_table()
    else :
        return "NO MOC DATA"

In [13]:
table = tablesWithMOC[0]
tapService = vo.dal.TAPService(table["url"])

data = tapService.search("SELECT TOP 10 * FROM " + table["name"]).to_table()
print(data)

granule_uid granule_gid   obs_id  ... science_case_id sc_pointing_mode
                                  ...                                 
----------- ----------- --------- ... --------------- ----------------
 VV0320_01C  calibrated VV0320_01 ...               2           MOSAIC
 VV0320_01G    geometry VV0320_01 ...               2           MOSAIC
 VV2878_02C  calibrated VV2878_02 ...               0             NULL
 VV2878_02G    geometry VV2878_02 ...               0             NULL
 VV2875_01C  calibrated VV2875_01 ...               0             NULL
 VV2875_01G    geometry VV2875_01 ...               0             NULL
 VV0324_06C  calibrated VV0324_06 ...               7            INERT
 VV0324_06G    geometry VV0324_06 ...               7            INERT
 VV0324_08C  calibrated VV0324_08 ...               7            INERT
 VV0324_08G    geometry VV0324_08 ...               7            INERT


## Import modules for Aladin and MOCs

In [14]:
import astropy.units as u
from astropy.coordinates import SkyCoord

from astroquery.cds import cds
from astroquery.vizier import Vizier

from mocpy import MOC

from ipyaladin import Aladin

## Get the data

In [15]:
table = tablesWithMOC[0]
tapService = vo.dal.TAPService(table["url"])

data = tapService.search("SELECT TOP 500 * FROM " + table["name"] + " WHERE coverage IS NOT NULL").to_table()
print(data)

granule_uid granule_gid   obs_id  ... science_case_id sc_pointing_mode
                                  ...                                 
----------- ----------- --------- ... --------------- ----------------
 VV0320_01C  calibrated VV0320_01 ...               2           MOSAIC
 VV0320_01G    geometry VV0320_01 ...               2           MOSAIC
 VV2878_02C  calibrated VV2878_02 ...               0             NULL
 VV2878_02G    geometry VV2878_02 ...               0             NULL
 VV2875_01C  calibrated VV2875_01 ...               0             NULL
 VV2875_01G    geometry VV2875_01 ...               0             NULL
 VV0324_06C  calibrated VV0324_06 ...               7            INERT
 VV0324_06G    geometry VV0324_06 ...               7            INERT
 VV0324_08C  calibrated VV0324_08 ...               7            INERT
 VV0324_08G    geometry VV0324_08 ...               7            INERT
        ...         ...       ... ...             ...              ...
 VV008

## Load Aladin and add the data that was fetched just above

Here we are using the hips of Venus

In [16]:
aladin = Aladin(
    coo_frame=data[0]["spatial_frame_type"],
    survey="http://voparis-srv-paris.obspm.fr/vo/planeto/hips/CDS_P_Venus_Magellan_C3-MDIR-2025m/"
)
aladin

Aladin(coo_frame='body', options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_sur…

In [17]:
aladin.add_table(data)

## Load the first MOC data

It needs to be converted to JSON

In [18]:
mocData = tapService.search("SELECT TOP 1 coverage FROM " + table["name"]).to_table()[0]["coverage"]

In [19]:
moc = MOC.from_str(mocData)

In [20]:
mocCenter = moc.barycenter()

In [21]:
jsonMoc = moc.serialize(format='json', optional_kw_dict=None, pre_v2=False)
aladin.add_moc_from_dict(jsonMoc, {'color' : 'red', 'opacity' : 0.5})
aladin.target = mocCenter.to_string()

## Intersection of MOCs

In [90]:
table = tablesWithMOC[0]
tapService = vo.dal.TAPService(table["url"])
query = 'SELECT * FROM '+table["name"]+' WHERE c1min >= 0.0 AND c1min <= 360.0 AND c1max >= 0.0 AND c1max <= 360.0 AND c2min >= 0.0 AND c2min <= 90.0 AND c2max >= 0.0 AND c2max <= 90.0 AND dataproduct_type LIKE \'%sc%\' AND local_time_min <= 20'

data = tapService.search(query).to_table()

In [91]:
print("granule_uid\tchannel_id\tc1min ; c1max\t\tc2min ; c2max\t\tra ; dec")
for e in data :
    print(e["granule_uid"],"\t",e["channel_id"], "\t", e["c1min"], ";", e["c1max"], "\t", e["c2min"], ";", e["c2max"],"\t",e["ra"],";",e["dec"])

granule_uid	channel_id	c1min ; c1max		c2min ; c2max		ra ; dec
VV0041_02C 	 VIRTIS_M_VIS 	 356.868 ; 357.605 	 7.18175 ; 69.9193 	 -999.999 ; -999.999
VV0047_00C 	 VIRTIS_M_VIS 	 187.423 ; 3.6804 	 4.82867 ; 89.6728 	 104.475 ; -62.5035
VV0058_00C 	 VIRTIS_M_VIS 	 203.97 ; 207.854 	 4.6971 ; 84.4626 	 105.0 ; -63.0
VV0062_00C 	 VIRTIS_M_VIS 	 205.277 ; 209.194 	 49.8892 ; 79.8392 	 102.218 ; -16.7112
VV0060_05C 	 VIRTIS_M_VIS 	 206.929 ; 210.947 	 4.31105 ; 84.6661 	 105.0 ; -63.0
VT0033_00C 	 VIRTIS_H 	 165.763 ; 345.301 	 6.09572 ; 89.6878 	 97.0 ; -63.0
VT0060_01C 	 VIRTIS_H 	 207.105 ; 209.219 	 17.0944 ; 56.5911 	 105.0 ; -63.0
VT0062_00C 	 VIRTIS_H 	 205.638 ; 209.208 	 54.6843 ; 77.9203 	 102.218 ; -16.7112
VT0041_03C 	 VIRTIS_H 	 357.249 ; 357.394 	 17.0473 ; 50.3341 	 -999.999 ; -999.999
VT0047_00C 	 VIRTIS_H 	 187.808 ; 189.872 	 9.29254 ; 57.4396 	 105.0 ; -63.0
VT0058_00C 	 VIRTIS_H 	 204.138 ; 206.249 	 16.853 ; 56.5184 	 105.0 ; -63.0
VI0041_02C 	 VIRTIS_M_IR 	 356.868 ; 3

In [92]:
colors = ["red","blue","green"]

In [93]:
mocGroups = {"VIRTIS_M_VIS":[], "VIRTIS_M_IR":[], "VIRTIS_H":[]}
for element in data :
    mocGroups[element["channel_id"]].append(MOC.from_str(element["coverage"]))

keys = list(mocGroups.keys())
for key in keys:
    if(len(mocGroups[key]) > 0):
        mocUnion = mocGroups[key][0]
        mocGroups[key].pop(0)
        for moc in mocGroups[key] :
            mocUnion = mocUnion.union(moc)
        mocGroups[key] = mocUnion
    else :
        mocGroups.pop(key)

In [94]:
aladin = Aladin(
    coo_frame=data[0]["spatial_frame_type"],
    survey="http://voparis-srv-paris.obspm.fr/vo/planeto/hips/CDS_P_Venus_Magellan_C3-MDIR-2025m/"
)
aladin

Aladin(coo_frame='body', options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_sur…

In [95]:
i = 0
for group, moc in mocGroups.items() :
    jsonMoc = moc.serialize(format='json', optional_kw_dict=None, pre_v2=False)
    aladin.add_moc_from_dict(jsonMoc, {'color' : colors[i], 'opacity' : 0.5, 'name' : group})
    i = i + 1

In [96]:
mocList = []
for moc in mocGroups.values() :
    mocList.append(moc)
    
if(len(mocGroups) > 0):
    mocIntersection = mocList[0]
    mocList.pop(0)
    for moc in mocList :
        mocIntersection = mocIntersection.intersection(moc)

In [97]:
results = []
for item in data :
    moc = MOC.from_str(item["coverage"])
    if not moc.intersection(mocIntersection).empty() :
        results.append(item)        

In [98]:
print(len(results), " results found:")
for item in results :
    print(item["granule_uid"])

15  results found:
VV0041_02C
VV0047_00C
VV0058_00C
VV0062_00C
VV0060_05C
VT0060_01C
VT0062_00C
VT0041_03C
VT0047_00C
VT0058_00C
VI0041_02C
VI0047_00C
VI0058_00C
VI0060_05C
VI0062_00C


In [100]:
jsonMoc = mocIntersection.serialize(format='json', optional_kw_dict=None, pre_v2=False)
aladin.add_moc_from_dict(jsonMoc, {'color' : 'yellow', 'opacity' : 0.5, 'name' : 'intersection'})